# **_<div class="alert alert-info"><font color='darkred'> Cartografiando las Lomas</font></div>_**

## 1. Importación de las bibliotecas necesarias

In [7]:
import ee
import numpy as np
import geemap as emap
import geemap.colormaps as cm
Map = emap.Map()
Map.add_basemap('ROADMAP')

## 2. Análisis de frecuencia de la vegetación en las Lomas

Para este análisis, es necesario visualizar el polígono de Lomas mapeadas previamente.

In [3]:
# Establecemos el poligono de lomas mapeadas
region = ee.FeatureCollection('users/geoyons/tugee/LimaProv')
mapLomas = ee.FeatureCollection('users/geoyons/tugee/Lomas_Modis')

In [9]:
# Definimos la coleccion
collection = 'MODIS/006/MOD13Q1'

bands = ['NDVI']

def maskSQA(image):
    mask = image.select('SummaryQA').eq(0).Or(image.select('SummaryQA').eq(1))
    imageMask = image.updateMask(mask)
    return imageMask

ini_date = ['2000-07-01','2001-07-01','2002-07-01','2003-07-01','2004-07-01','2005-07-01','2006-07-01','2007-07-01',
           '2008-07-01','2009-07-01','2010-07-01','2011-07-01','2012-07-01','2013-07-01','2014-07-01','2015-07-01',
           '2016-07-01','2017-07-01','2018-07-01','2019-07-01','2020-07-01']

end_date = ['2000-10-30','2001-10-30','2002-10-30','2003-10-30','2004-10-30','2005-10-30','2006-10-30','2007-10-30',
           '2008-10-30','2009-10-30','2010-10-30','2011-10-30','2012-10-30','2013-10-30','2014-10-30','2015-10-30',
           '2016-10-30','2017-10-30','2018-10-30','2019-10-30','2020-10-30']

freqLomas = ee.Image(0)
for i in np.arange(0, len(ini_date), 1):
    
    col = ee.ImageCollection(collection)\
        .filterDate(ini_date[i],end_date[i])\
        .filterBounds(mapLomas)\
        .map(maskSQA)\
        .reduce(ee.Reducer.max())\
        .select([0], bands)
    freq = col.select('NDVI').divide(10000).gte(0.15).unmask(0).clip(mapLomas)
    freqLomas = freqLomas.add(freq)

freqLomas.getInfo()

palette = cm.palettes.viridis
# Definimos los parametros de visualizacion
visParams = {
  'min': 0,
  'max': 21,
  'palette': palette
}

# Centramos el mapa y visualizamos
Map.centerObject(mapLomas, 9)
Map.addLayer(region, {}, 'Provincia de Lima', True)
Map.addLayer(mapLomas, {}, 'Lomas de la Prov. Lima', True)
Map.addLayer(freqLomas, visParams, 'Freq. de Vet. Lomas')
Map.add_colorbar(visParams, label="Frecuencia de Aparición de Lomas", layer_name="SRTM DEM")
Map

Map(bottom=70278.0, center=[-12.109113041673206, -76.85975617641348], controls=(WidgetControl(options=['positi…

### 2.1. Descarga del raster de probabilidades

In [ ]:
# Descargar directamente a Google Drive
task = ee.batch.Export.image(freqLomas.divide(21), 
                              'PROB_VEGT_LOMAS_2000-2020_MODIS', 
                              {'region': region.geometry(),
                               'scale': 250,
                               'maxPixels': 1e13,
                               'folder': 'MOD13Q1_MODIS',
                               'fileFormat': 'GeoTIFF',
                               'crs': 'EPSG:4326'})
task.start()